## A module for data preparation of material properties for the case study

In [33]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Create a dataframe for parameters needed

All components require molecular weight. If a component is burnable(EGEN=1), LHV is required. If a component is a trace component, the material property other than molecular weight is allowed to be empty. Otherwise, all properties are needed.

In [34]:
compounds = [f'C{n}A' for n in range(1, 32)] + ['H2', 'LDPE', 'HDPE', 'PP', 'WASTE']

# Create a DataFrame with empty values (NaN) and the compounds as the row index
df = pd.DataFrame(index=compounds, columns=['MW', 'LHV', 'CN', 'RON', 'rho', 'mu', 'TB', 'trace', 'EGEN'])
df;

In [35]:
material = pd.read_csv("parameters/material.csv")
trace_dict = {}
egen_dict = {}

for index, row in material.iterrows():
    trace_dict[row["Code"]] = row["trace"]
    egen_dict[row["Code"]] = row["Egen"]

trace_dict, egen_dict;

In [36]:
for key, val in trace_dict.items():
    df.loc[key]["trace"] = val
    df.loc[key]["EGEN"] = egen_dict[key]

df.loc["WASTE", "trace"] = 1
df.loc["WASTE", "EGEN"] = 0
df

,MW,LHV,CN,RON,rho,mu,TB,trace,EGEN
C1A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
C2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
C3A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C4A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C6A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C7A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C8A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C9A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
C10A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


### The density, boiling point, and molecular weight are mostly obtained from [Engineering toolbox](https://www.engineeringtoolbox.com/hydrocarbon-boiling-melting-flash-autoignition-point-density-gravity-molweight-d_1966.html.)

In [37]:
def fetch_hydrocarbon_data(url):
    # Send a request to the URL
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract data - this part depends on the structure of the webpage
        # You'll need to inspect the page to determine how the data is structured
        # and how best to extract it
        data = []
        for table in soup.find_all("table"):
            # Extract data from each table row
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                row_data = [cell.text.strip() for cell in cells]
                data.append(row_data)
        return data
    else:
        return "Failed to retrieve data"

# URL of the page to scrape
url = "https://www.engineeringtoolbox.com/hydrocarbon-boiling-melting-flash-autoignition-point-density-gravity-molweight-d_1966.html"
hydrocarbon_raw_data = fetch_hydrocarbon_data(url)
# print(hydrocarbon_raw_data)

In [38]:
hydrocarbon = pd.DataFrame(hydrocarbon_raw_data[1:-1],  columns=["class","IUPAC name","Common name", "#C", "#H","MW", "melting   pooint(C)", "boiling point(C)", "rho(20C)[g/ml]",
                                                                 "flash point(C)", "autoignition temp(C)"])
hydrocarbon

,class,IUPAC name,Common name,#C,#H,MW,melting pooint(C),boiling point(C),rho(20C)[g/ml],flash point(C),autoignition temp(C)
0,N-alkane,Methane,,1,4,16.04,-183,-162,-,-135,595
1,N-alkane,Ethane,,2,6,30.07,-183,-89,-,-135,515
2,N-alkane,Propane,,3,8,44.09,-188,-42,0.498,-104,470
3,N-alkane,N-butane,Butane,4,10,58.12,-138,-1,0.577,-60,365
4,N-alkane,N-pentane,Pentane,5,12,72.15,-130,36,0.630,-49,260
...,...,...,...,...,...,...,...,...,...,...,...
217,,,,,,,,,,,
218,Tetraaromatics,Benzophenanthrene,,18,12,228.28,256,448,1.274,,
219,Tetraaromatics,Chrysene,"1,2-Benzophenanthrene",18,12,228.28,255,448,1.274,,
220,Tetraaromatics,Triphenylene,"IsoChrysene, 1,2,3,4-Dibenzonaphthalene",18,12,228.28,196,438,1.302,,


In [39]:
alkane_prop1 = hydrocarbon[0:31]
alkane_indices = [f"C{i}A" for i in range(1, 32)]
alkane_prop1.index = alkane_indices
alkane_prop1

,class,IUPAC name,Common name,#C,#H,MW,melting pooint(C),boiling point(C),rho(20C)[g/ml],flash point(C),autoignition temp(C)
C1A,N-alkane,Methane,,1,4,16.04,-183,-162,-,-135,595
C2A,N-alkane,Ethane,,2,6,30.07,-183,-89,-,-135,515
C3A,N-alkane,Propane,,3,8,44.09,-188,-42,0.498,-104,470
C4A,N-alkane,N-butane,Butane,4,10,58.12,-138,-1,0.577,-60,365
C5A,N-alkane,N-pentane,Pentane,5,12,72.15,-130,36,0.630,-49,260
C6A,N-alkane,N-hexane,Hexane,6,14,86.17,-95,69,0.664,<-20,230
C7A,N-alkane,N-heptane,Heptane,7,16,100.20,-91,98,0.683,-7,220
C8A,N-alkane,N-octane,Octane,8,18,114.22,-57,126,0.702,12,205
C9A,N-alkane,N-nonane,Nonane,9,20,128.25,-53,151,0.719,31,205
C10A,N-alkane,N-decane,Decane,10,22,142.28,-30,174,0.730,46,200


In [40]:
alkanes_mw = alkane_prop1["MW"]
alkanes_bp = alkane_prop1["boiling point(C)"]
alkanes_rho = alkane_prop1["rho(20C)[g/ml]"]
alkanes_mw_dict = {index: float(value) for index, value in alkanes_mw.items()}
alkanes_mw_dict["H2"] = 2.016
alkanes_mw_dict["LDPE"] = 28.05
alkanes_mw_dict["HDPE"] = 28.05
alkanes_mw_dict["PP"] = 42.081

In [41]:
alkanes_bp_dict = {index: float(value) for index, value in alkanes_bp.items()}
alkanes_bp_dict["H2"] = -252.8

In [42]:
alkanes_rho_dict = {index: float(value) for index, value in alkanes_rho.items() if not index in{"C1A", "C2A"}}

In [43]:
def write_series_to_gams(series, filename, scale=1, dec=6):
    indent = f"\t"
    with open(filename, 'w') as file:
        file.write(f"/\n")
        for index, value in series.items():
            # Writing the index and value in the required GAMS format
            # Assuming GAMS format is something like "i / value /"
            if not np.isnan(value):
                file.write(f"{indent}{index}  {round(float(value)*scale, dec)} \n")
        file.write(f"/\n")

# rite_series_to_gams(alkanes_mw_dict, "mw_pars.txt", scale=1e-3)

In [44]:
# write_series_to_gams(alkanes_bp_dict, "bp_pars.txt", scale=1)

In [45]:
# write_series_to_gams(alkanes_rho_dict, "rho_pars.txt", scale=1e3)

In [46]:
for key, val in alkanes_mw_dict.items():
    df.loc[key]["MW"] = val

for key, val in alkanes_bp_dict.items():
    df.loc[key]["TB"] = val

for key, val in alkanes_rho_dict.items():
    df.loc[key]["rho"] = val

df

,MW,LHV,CN,RON,rho,mu,TB,trace,EGEN
C1A,16.04,NaN,NaN,NaN,NaN,NaN,-162.0,1,1
C2A,30.07,NaN,NaN,NaN,NaN,NaN,-89.0,1,1
C3A,44.09,NaN,NaN,NaN,0.498,NaN,-42.0,0,1
C4A,58.12,NaN,NaN,NaN,0.577,NaN,-1.0,0,1
C5A,72.15,NaN,NaN,NaN,0.63,NaN,36.0,0,1
C6A,86.17,NaN,NaN,NaN,0.664,NaN,69.0,0,1
C7A,100.2,NaN,NaN,NaN,0.683,NaN,98.0,0,1
C8A,114.22,NaN,NaN,NaN,0.702,NaN,126.0,0,1
C9A,128.25,NaN,NaN,NaN,0.719,NaN,151.0,0,1
C10A,142.28,NaN,NaN,NaN,0.73,NaN,174.0,0,1


### The data for viscosity and LHV are obtained from Aspen Plus

The LHV is calculated by considering the heat of reaction.

In [47]:
aspen_data = pd.read_csv("parameters/viscosity.csv", index_col=0)
aspen_data.loc["mu"],  aspen_data.loc["vapor"], aspen_data.loc["liquid"];

In [48]:
viscosity_dict = {index: float(value) for index, value in aspen_data.loc["mu"].items()}
# convert Pa*S to mm^2/s, multiply by 1000
write_series_to_gams(viscosity_dict, "mu_pars.txt", scale=1e3)
viscosity_dict;

In [49]:
vapor_dict = {index: float(value) for index, value in aspen_data.loc["vapor"].items()}
liquid_dict = {index: float(value) for index, value in aspen_data.loc["liquid"].items()}
keys = liquid_dict.keys()
dH = {}
for key in keys:
    if key == "H2":
        pass
    elif key in {"C1A", "C2A", "C3A", "C4A"}:
        dH[key] = vapor_dict[key]
    else:
        dH[key] = liquid_dict[key]
dH;

In [50]:
Hf = {"H2O": -241.917, "O2":0, "CO2":-393.552} # liquid H2O, vapor for O2, CO2 # kj/mol

In [51]:
def calculate_lhv(fuel, dH, Hf):
    # Parse the number of carbon (C) and hydrogen (H) atoms from the fuel name
    carbon_atoms = int(fuel[1: fuel.find('A')])
    hydrogen_atoms = (carbon_atoms * 2) + 2  # General formula for alkanes is CnH(2n+2)

    # Balance the combustion reaction (assuming complete combustion)
    # Stoichiometry of oxygen required for complete combustion:
    # Each C requires 1 mole of O2 for CO2 and each H2 requires 0.5 mole of O2 for H2O
    o2_moles = carbon_atoms + hydrogen_atoms / 4
    co2_moles = carbon_atoms
    h2o_moles = hydrogen_atoms / 2

    # Calculate the heat of the reaction based on stoichiometry and heats of formation
    # ΔH_reaction = Σ(ΔHf_products) - Σ(ΔHf_reactants)
    # For reactants, we consider the fuel and O2; for products, CO2 and H2O
    # Note: O2 is the reference with ΔHf = 0, so it's not included in the calculation
    heat_of_reaction = (co2_moles * Hf['CO2'] + h2o_moles * Hf['H2O']) - o2_moles * Hf['O2'] - dH[fuel]

    # LHV calculation: Since the reaction is exothermic, heat of reaction will be negative, so we take the negative of it
    # to represent the energy released
    LHV = -heat_of_reaction

    return LHV  # This will be the LHV in kJ/mol assuming ΔHf values are also in kJ/mol

In [52]:
dfHG = {"H2O": -242.00, "O2":-447.78, "CO2":-393.80} # liquid H2O, vapor for O2, CO2
lhv_data = {key: calculate_lhv(key, dH, Hf) for key in dH.keys()}
lhv_data["H2"] = 241722
# write_series_to_gams(lhv_data, "lhv_pars.txt", scale=1e3)
lhv_data;

In [53]:
for key, val in lhv_data.items():
    df.loc[key]["LHV"] = val

for key, val in viscosity_dict.items():
    df.loc[key]["mu"] = val

df

,MW,LHV,CN,RON,rho,mu,TB,trace,EGEN
C1A,16.04,802.8471,NaN,NaN,NaN,0.000016,-162.0,1,1
C2A,30.07,1428.9758,NaN,NaN,NaN,0.000016,-89.0,1,1
C3A,44.09,2043.53,NaN,NaN,0.498,0.000089,-42.0,0,1
C4A,58.12,2657.816,NaN,NaN,0.577,0.000152,-1.0,0,1
C5A,72.15,3245.91,NaN,NaN,0.63,0.000222,36.0,0,1
C6A,86.17,3856.423,NaN,NaN,0.664,0.000306,69.0,0,1
C7A,100.2,4466.551,NaN,NaN,0.683,0.000405,98.0,0,1
C8A,114.22,5076.26,NaN,NaN,0.702,0.000528,126.0,0,1
C9A,128.25,5687.282,NaN,NaN,0.719,0.000679,151.0,0,1
C10A,142.28,6297.381,NaN,NaN,0.73,0.000875,174.0,0,1


### Lastly, RON and CN are provided.

In [54]:
# RON, CN
fuel_cn_dict = dict()
fuel_ron_dict = dict()
fuel_prop = pd.read_csv("parameters/fuel_prop.csv")
for index, row in fuel_prop.iterrows():
    fuel_cn_dict[row["Code"]] = row["CN"]
    fuel_ron_dict[row["Code"]] = row["RON"]

# write_series_to_gams(fuel_cn_dict, "fuel_cn_dict.txt", scale=1)
# write_series_to_gams(fuel_ron_dict, "fuel_ron_dict.txt", scale=1)

for key, val in fuel_cn_dict.items():
    df.loc[key]["CN"] = val

for key, val in fuel_ron_dict.items():
    df.loc[key]["RON"] = val

df

,MW,LHV,CN,RON,rho,mu,TB,trace,EGEN
C1A,16.04,802.8471,0,127,NaN,0.000016,-162.0,1,1
C2A,30.07,1428.9758,-20,111,NaN,0.000016,-89.0,1,1
C3A,44.09,2043.53,-20,111,0.498,0.000089,-42.0,0,1
C4A,58.12,2657.816,21,94,0.577,0.000152,-1.0,0,1
C5A,72.15,3245.91,30,62,0.63,0.000222,36.0,0,1
C6A,86.17,3856.423,45,19,0.664,0.000306,69.0,0,1
C7A,100.2,4466.551,56,0,0.683,0.000405,98.0,0,1
C8A,114.22,5076.26,58,0,0.702,0.000528,126.0,0,1
C9A,128.25,5687.282,61,0,0.719,0.000679,151.0,0,1
C10A,142.28,6297.381,66,0,0.73,0.000875,174.0,0,1


### Now write data to gams.

In [55]:
write_series_to_gams(df["MW"].to_dict(), "mw_pars.txt", scale=1e-3)
write_series_to_gams(df["LHV"].to_dict(), "lhv_pars.txt", scale=1e3)
write_series_to_gams(df["CN"].to_dict(), "cn_pars.txt", scale=1)
write_series_to_gams(df["RON"].to_dict(), "ron_pars.txt", scale=1)
write_series_to_gams(df["rho"].to_dict(), "rho_pars.txt", scale=1e3)
write_series_to_gams(df["mu"].to_dict(), "mu_pars.txt", scale=1e3)
write_series_to_gams(df["TB"].to_dict(), "bp_pars.txt", scale=1)
write_series_to_gams(df["trace"].to_dict(), "trace_pars.txt", scale=1)
write_series_to_gams(df["EGEN"].to_dict(), "egen_pars.txt", scale=1)

In [56]:
def write_list_to_file(strings, filename, n, separator=", "):
    with open(filename, "w") as f:
        f.write(f"/\n")
        for i in range(0, len(strings), n):
            # Check if it's the last line
            last_line = i + n >= len(strings)
            if not last_line:
                separator_with_newline = f"{separator}\n"
            else:
                separator_with_newline = "\n"
            f.write(f"\t")
            f.write(separator.join(strings[i:i+n]) + separator_with_newline)
        f.write(f"/")

n = 8
filename = "subsets.txt"
separator = ","

In [57]:
bp30 = []
bp150 = []
bp160 = []
bp215 = []
bp300 = []
bp360 = []
kegen = []
kfuel = []
ktrace =[]

lambda_tb = {}

for key, val in df["TB"].to_dict().items():
    lambda_tb[key] = val
    if val < 30:
        bp30.append(key)
    if val < 150:
        bp150.append(key)
    if val < 160:
        bp160.append(key)
    if val < 215:
        bp215.append(key)
    if val < 300:
        bp300.append(key)
    if val < 360:
        bp360.append(key)

for key, val in df["EGEN"].to_dict().items():
    if val == 0:
        kegen.append(key)

for key, val in df["trace"].to_dict().items():
    if val == 0:
        kfuel.append(key)
    else:
        ktrace.append(key)

In [58]:
def write_dict_to_gams(data, filename, scale=1, dec=6):
    indent = f"\t"
    with open(filename, 'w') as file:
        file.write(f"/\n")
        for index, value in data.items():
            # Writing the index and value in the required GAMS format
            # Assuming GAMS format is something like "i / value /"
            if not np.isnan(value):
                file.write(f"{indent}{index}  {round(float(value)*scale, dec)} \n")
        file.write(f"/\n")

In [59]:
write_list_to_file(bp30, "bp30.txt", 8, ",")
write_list_to_file(bp150, "bp150.txt", 8, ",")
write_list_to_file(bp160, "bp160.txt", 8, ",")
write_list_to_file(bp215, "bp215.txt", 8, ",")
write_list_to_file(bp300, "bp300.txt", 8, ",")
write_list_to_file(bp360, "bp360.txt", 8, ",")
write_list_to_file(kegen, "kegen.txt", 8, ",")
write_list_to_file(kfuel, "kfuel.txt", 8, ",")
write_list_to_file(ktrace, "ktrace.txt", 8, ",")

### The chemeo data collector, currently not used.

In [60]:
# chemeo_df = pd.read_csv("parameters/chemeo.csv")
# chemeo_urls = {}
# for index, row in chemeo_df.iterrows():
#     chemeo_urls[row["code"]] = row["source"]
# # chemeo_urls

In [61]:
# # The URLs
#
# prop_data = {}
# for k, val in chemeo_urls.items():
#     properties_dict = {}
#     url = val
#
#     response = requests.get(url)
#     # Check if the request was successful
#     if response.status_code == 200:
#         # Parse the HTML content of the page
#         soup = BeautifulSoup(response.content, 'html.parser')
#
#         # Find and extract the desired data
#         properties_table = soup.find('table', class_='props details')
#         # Iterate over each row in the table
#         print(f"retrieving data {k}...")
#         for row in properties_table.find_all('tr'):
#             # Find all columns in the row
#             cols = row.find_all('td')
#
#             # Extract text and assign to dictionary (assuming two columns per row)
#             if len(cols) == 4:
#                 key = cols[0].text.strip()
#                 value = cols[1].text.strip()
#                 properties_dict[key] = value
#
#         prop_data[k] = properties_dict["ΔfH°gas"]
#     else:
#         print('Failed to retrieve the webpage')
#         prop_data[k] = None
#
# # prop_data

In [62]:
# prop_data
# # Processing the values to cast them to float and drop the "±" part
# processed_data = {key: float(value.split(' ± ')[0]) for key, value in prop_data.items()}
# processed_data

In [63]:
# dfHG = {"H2O": -242.00, "O2":-447.78, "CO2":-393.80} # liquid H2O, vapor for O2, CO2
# lhv_data = {key: calculate_lhv(key, processed_data, Hf) for key in processed_data.keys()}
# lhv_data